In [3]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="google.protobuf")


In [4]:
import os
import sys
os.chdir('/home/victorhuang/projects/gtx')
os.environ.pop("LD_LIBRARY_PATH", None)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'


In [5]:
import tensorflow as tf
print(tf.test.is_built_with_cuda())
print(tf.config.list_physical_devices("GPU"))


True
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [6]:
import scipy.io as sio
import mat73
import numpy as np
import matplotlib.pyplot as plt

In [7]:
import keras
import tensorflow as tf
from keras.saving import register_keras_serializable

In [8]:
print("Python:", sys.executable)
print("TF version:", tf.__version__)
print("CUDA built with:", tf.sysconfig.get_build_info().get("cuda_version"))
print("cuDNN built with:", tf.sysconfig.get_build_info().get("cudnn_version"))

Python: /home/victorhuang/miniconda3/envs/gtx_tf/bin/python
TF version: 2.20.0
CUDA built with: 12.5.1
cuDNN built with: 9


In [9]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        print(gpus)
        tf.config.set_visible_devices(gpus[0], 'GPU')
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# Model

## unzip model

In [4]:
!tar -xvzf code_tf/aws_ckpt/8000_1024_mix_unet/model.tar.gz -C code_tf/aws_ckpt/8000_1024_mix_unet/

tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
loss.log
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
model.keras


## test model

In [11]:
@register_keras_serializable(package="custom")
def tumor_mae(y_true, y_pred):
    mask = tf.not_equal(y_true, 0.0)
    err  = tf.abs(y_true - y_pred)
    masked_err = tf.boolean_mask(err, mask)
    return tf.cond(tf.size(masked_err) > 0,
                   lambda: tf.reduce_mean(masked_err),
                   lambda: tf.constant(0.0, dtype=y_true.dtype))

In [12]:
model = keras.models.load_model("code_tf/aws_ckpt/8000_1024_mix_unet/model.keras", safe_mode=True)

input_1 = tf.constant(np.ones((1, 101, 101, 2), dtype=np.float32))
input_2 = tf.constant(np.ones((1, 101, 101, 6, 1), dtype=np.float32))

output = model([input_1, input_2], training=False)

print(output[0].shape)
print(output[1].shape)

I0000 00:00:1758125854.551841    4594 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5520 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4070 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


(1, 101, 101)
(1, 101, 101)
